### 금융문자 분석
- 데이터 제공이 안된다.... 
- 베이스 라인 코드 따라쳐보고 의미 파악 하는 방향으로..

In [1]:
import pandas as pd
import numpy as np
import random
from pandas import DataFrame
from collections import Counter

from tqdm import tqdm # 시간 측정

from sklearn.feature_extraction.text import CountVectorizer # 벡터라이저 모델
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

/home/loveactualry/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [ ]:
# colab 에서 사용 하는 경우 명령어.
# colab을 써볼까??? 
# 환경 설정 걱정 안해도 될 것 같은데...
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# 변수에 값들 세주기...
Counter(train['smishing'])
# 결과를 보면 스미싱과 아닌 것의 개수가 Counter({0: 277242, 1: 18703})
# 비율 차이가 좀 있다.. 
# 산탄데르 문제와 비슷..?

In [ ]:
# 램을 덜 차지하게 비율을 찾아보는 과정
# under-sampling 방법
19646/train.shape[0]

In [ ]:
random.seed(2019) # 시드번호
train_nsm_list=list(train[train['smishg']!=1].index
# 0인 인덱스만 뽑아서 리스트를 만든다.
train_nsmishing=random.sample(train_nsm_list, 11750)
# 랜덤으로 11750개 샘플 뽑아서 저장.

In [ ]:
random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [ ]:
train_xx=train.iloc[train_smishing+train_nsmishing,:]\
.reset_index(drop=True)

# reset index  보기